In [ ]:
import asyncio
import time

### Where to use it?
1. Multiprocessing: CPU intensive work - Memory sharing is hard
2. Multithreading: Thread based workloads having common shared memory
3. Async IO: You need to wait a lot in between for operations like API call, file reads

###  Coroutines

- You can `asyncio.run()` in Jupyter notebooks

- Calling async functions return a `coroutine` object, this can then be awaited or passed to `asyncio.
- Also, these coroutines when manually awaited or run, dont run parallely ( totally the code here takes 4 seconds, it should have taken only 2 if run paralelyl)

In [ ]:
async def do_some_task(id, delay):
    print(f"Performing task {id} with delay of {delay} after this...")
    asyncio.sleep()
    print(f"Performed task {id} after delay of {delay}")
    return {"data": id}

In [ ]:
async def main():
    start_time = time.time()

    task1 = do_some_task(1, 2) # 2 seconds delay
    task2 = do_some_task(2, 2) # 2 seconds delay

    res1 = await task1
    print(res1)

    res2 = await task2
    print(res2)

    print(f"Total time taken: {time.time()-start_time}")


In [ ]:
asyncio.run(main())

"""
Performing task 1 with delay of 2 after this...
Performed task 1 after delay of 2
{'data': 1}
Performing task 2 with delay of 2 after this...
Performed task 2 after delay of 2
{'data': 2}
Total time taken: 4.014990329742432
"""

### Tasks

- Tasks can run paralelly ( just like threads )
- BETER, use `TaskGroups`

In [6]:
async def main():
    start_time = time.time()

    task1 = asyncio.create_task(do_some_task(1, 2))
    task2 = asyncio.create_task(do_some_task(2, 2))

    res1 = await task1
    res2 = await task2

    print(res1)
    print(res2)

    print(f"Total time taken: {time.time()-start_time}")

In [ ]:
async def main():
    start_time = time.time()

    tasks = []
    async with asyncio.TaskGroup() as tg:
        tasks.append(tg.create_task(do_some_task(1, 2)))
        tasks.append(tg.create_task(do_some_task(2, 2)))

    results = [task.result() for task in tasks]
    print(results)

    print(f"Total time taken: {time.time()-start_time}")


In [ ]:
"""
Performing task 1 with delay of 2 after this...
Performing task 2 with delay of 2 after this...
Performed task 1 after delay of 2
Performed task 2 after delay of 2
{'data': 1}
{'data': 2}
Total time taken: 2.008986234664917
"""

### TaskGroups with synchronized locks


In [ ]:
import asyncio
import time

counter = 0
lock = asyncio.Lock()

async def do_some_task(id, delay):
    global counter
    async with lock:
        print(f"Performing task {id} with delay of {delay} after this...")
        await asyncio.sleep(delay)
        print(f"Performed task {id} after delay of {delay}")
        return {"data": id}


async def main():
    start_time = time.time()

    tasks = []
    async with asyncio.TaskGroup() as tg:
        tasks.append(tg.create_task(do_some_task(1, 2)))
        tasks.append(tg.create_task(do_some_task(2, 2)))

    results = [task.result() for task in tasks]
    print(results)

    print(f"Total time taken: {time.time()-start_time}")



asyncio.run(main())
"""
Performing task 1 with delay of 2 after this...
Performed task 1 after delay of 2
Performing task 2 with delay of 2 after this...
Performed task 2 after delay of 2
[{'data': 1}, {'data': 2}]
Total time taken: 4.0154571533203125
"""

### Thread Events

### `threading.Event()` options
- Initially, the event is not set (False)
1. `.set()` - Sets the flag to True (signals all waiting threads)
2. `.clear()` - Resets the flag to False
3. `.is_set()` - Returns True if flag is set, False otherwise
4. `.wait()` - Blocks until the flag becomes True

In [ ]:
import threading
import time

# Scenario: Worker threads wait for a "ready" signal before starting work

ready_signal = threading.Event()
stop_signal = threading.Event()

def worker(worker_id):
    print(f"Worker {worker_id}: Waiting for ready signal...")
    ready_signal.wait()  # Block until event is set
    print(f"Worker {worker_id}: Started working!")
    
    while not stop_signal.is_set():
        print(f"Worker {worker_id}: Processing...")
        time.sleep(1)
    
    print(f"Worker {worker_id}: Stopped!")

# Create worker threads
threads = []
for i in range(3):
    t = threading.Thread(target=worker, args=(i,))
    t.start()
    threads.append(t)

print("Main: Workers created, waiting 2 seconds...")
time.sleep(2)

print("Main: Signaling workers to start!")
ready_signal.set()  # Release all waiting workers

time.sleep(3)

print("Main: Signaling workers to stop!")
stop_signal.set()  # Tell workers to stop

# Wait for all threads to finish
for t in threads:
    t.join()

print("Main: All workers finished!")